In [1]:
import pandas as pd
import sqlalchemy as sql

In [2]:
nrw_2022 = pd.read_csv("data/nrw_2022.csv")
nrw_2022.head(20)
nrw_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8832 entries, 0 to 8831
Data columns (total 10 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   Einzelplan                          8832 non-null   int64 
 1   Kapitel                             8832 non-null   int64 
 2   Gruppe                              8832 non-null   int64 
 3   Zählnummer                          8832 non-null   int64 
 4   Funktion                            8832 non-null   int64 
 5   Ansatz 2022 in Euro                 8832 non-null   object
 6   Ansatz 2021 in Euro                 8832 non-null   object
 7   Ist 2020 in Euro                    8832 non-null   object
 8   Verpflichtungsermächtigung in Euro  8832 non-null   object
 9   Zweckbestimmung                     8832 non-null   object
dtypes: int64(5), object(5)
memory usage: 690.1+ KB


In [3]:
nrw_2022.columns = nrw_2022.columns.str.replace(" ", "_")
nrw_2022.columns = nrw_2022.columns.str.lower()
nrw_2022.rename(columns={"ansatz_2022_t€":"ansatz_2022_euro"}, inplace=True)
nrw_2022.tail()

,einzelplan,kapitel,gruppe,zählnummer,funktion,ansatz_2022_in_euro,ansatz_2021_in_euro,ist_2020_in_euro,verpflichtungsermächtigung_in_euro,zweckbestimmung
8827,20,900,636,20,18,0,0,"0,00",0,Zusätzliche Alters- und Hinterbliebenenversorg...
8828,20,900,637,0,18,10.000,10.000,"1.040,14",0,Erstattungen von Versorgungsbezügen an Zweckve...
8829,20,900,671,0,18,10.000,10.000,"0,00",0,Sonstige Erstattungen von Versorgungsbezügen
8830,20,900,919,10,851,200.000.000,200.000.000,"273.652.769,81",0,"Zuführung an das Sondervermögen ""Pensionsfonds..."
8831,20,900,919,20,851,5.000.000,5.000.000,"5.008.001,71",0,Zuführung der von Dritten gezahlten Versorgung...


In [4]:
nrw_2022.rename(columns = {'einzelplan':'ep', 'zählnummer':'counter', }, inplace = True)
nrw_2022.head()

,ep,kapitel,gruppe,counter,funktion,ansatz_2022_in_euro,ansatz_2021_in_euro,ist_2020_in_euro,verpflichtungsermächtigung_in_euro,zweckbestimmung
0,1,10,119,1,11,20.000,40.000,"16.918,46",0,Vermischte Einnahmen
1,1,10,124,1,11,10.000,40.000,"7.524,00",0,Mieten und Pachten
2,1,10,232,0,11,0,0,"0,00",0,Erstattung von Verwaltungsausgaben durch die L...
3,1,10,236,0,11,0,0,"0,00",0,Erstattungen von Verwaltungsausgaben von Sozia...
4,1,10,261,0,11,2.000,2.000,"0,00",0,Schuldendiensthilfen und Erstattungen von Verw...


In [5]:
nrw_2022.drop('ist_2020_in_euro', inplace=True, axis=1)

In [6]:
nrw_2022.drop('ansatz_2021_in_euro', inplace=True, axis=1)
nrw_2022.head()


,ep,kapitel,gruppe,counter,funktion,ansatz_2022_in_euro,verpflichtungsermächtigung_in_euro,zweckbestimmung
0,1,10,119,1,11,20.000,0,Vermischte Einnahmen
1,1,10,124,1,11,10.000,0,Mieten und Pachten
2,1,10,232,0,11,0,0,Erstattung von Verwaltungsausgaben durch die L...
3,1,10,236,0,11,0,0,Erstattungen von Verwaltungsausgaben von Sozia...
4,1,10,261,0,11,2.000,0,Schuldendiensthilfen und Erstattungen von Verw...


In [7]:
nrw_2022.drop('verpflichtungsermächtigung_in_euro', inplace=True, axis=1)
nrw_2022.head()
nrw_2022.drop('funktion', inplace=True, axis=1)

In [8]:
nrw_2022

,ep,kapitel,gruppe,counter,ansatz_2022_in_euro,zweckbestimmung
0,1,10,119,1,20.000,Vermischte Einnahmen
1,1,10,124,1,10.000,Mieten und Pachten
2,1,10,232,0,0,Erstattung von Verwaltungsausgaben durch die L...
3,1,10,236,0,0,Erstattungen von Verwaltungsausgaben von Sozia...
4,1,10,261,0,2.000,Schuldendiensthilfen und Erstattungen von Verw...
...,...,...,...,...,...,...
8827,20,900,636,20,0,Zusätzliche Alters- und Hinterbliebenenversorg...
8828,20,900,637,0,10.000,Erstattungen von Versorgungsbezügen an Zweckve...
8829,20,900,671,0,10.000,Sonstige Erstattungen von Versorgungsbezügen
8830,20,900,919,10,200.000.000,"Zuführung an das Sondervermögen ""Pensionsfonds..."


In [9]:
nrw_2022.insert(loc=6,
          column='jahr',
          value=2022)

nrw_2022.head()

,ep,kapitel,gruppe,counter,ansatz_2022_in_euro,zweckbestimmung,jahr
0,1,10,119,1,20.000,Vermischte Einnahmen,2022
1,1,10,124,1,10.000,Mieten und Pachten,2022
2,1,10,232,0,0,Erstattung von Verwaltungsausgaben durch die L...,2022
3,1,10,236,0,0,Erstattungen von Verwaltungsausgaben von Sozia...,2022
4,1,10,261,0,2.000,Schuldendiensthilfen und Erstattungen von Verw...,2022


In [10]:
nrw_2022.insert(loc=7,
          column='state_id',
          value=10)

nrw_2022.head()

,ep,kapitel,gruppe,counter,ansatz_2022_in_euro,zweckbestimmung,jahr,state_id
0,1,10,119,1,20.000,Vermischte Einnahmen,2022,10
1,1,10,124,1,10.000,Mieten und Pachten,2022,10
2,1,10,232,0,0,Erstattung von Verwaltungsausgaben durch die L...,2022,10
3,1,10,236,0,0,Erstattungen von Verwaltungsausgaben von Sozia...,2022,10
4,1,10,261,0,2.000,Schuldendiensthilfen und Erstattungen von Verw...,2022,10


In [11]:
nrw_2022

,ep,kapitel,gruppe,counter,ansatz_2022_in_euro,zweckbestimmung,jahr,state_id
0,1,10,119,1,20.000,Vermischte Einnahmen,2022,10
1,1,10,124,1,10.000,Mieten und Pachten,2022,10
2,1,10,232,0,0,Erstattung von Verwaltungsausgaben durch die L...,2022,10
3,1,10,236,0,0,Erstattungen von Verwaltungsausgaben von Sozia...,2022,10
4,1,10,261,0,2.000,Schuldendiensthilfen und Erstattungen von Verw...,2022,10
...,...,...,...,...,...,...,...,...
8827,20,900,636,20,0,Zusätzliche Alters- und Hinterbliebenenversorg...,2022,10
8828,20,900,637,0,10.000,Erstattungen von Versorgungsbezügen an Zweckve...,2022,10
8829,20,900,671,0,10.000,Sonstige Erstattungen von Versorgungsbezügen,2022,10
8830,20,900,919,10,200.000.000,"Zuführung an das Sondervermögen ""Pensionsfonds...",2022,10


In [12]:
nrw_2022 = nrw_2022[['ep', 'kapitel', 'zweckbestimmung', 'ansatz_2022_in_euro', 'gruppe', 'counter', 'jahr', 'state_id']]
nrw_2022.head()

,ep,kapitel,zweckbestimmung,ansatz_2022_in_euro,gruppe,counter,jahr,state_id
0,1,10,Vermischte Einnahmen,20.000,119,1,2022,10
1,1,10,Mieten und Pachten,10.000,124,1,2022,10
2,1,10,Erstattung von Verwaltungsausgaben durch die L...,0,232,0,2022,10
3,1,10,Erstattungen von Verwaltungsausgaben von Sozia...,0,236,0,2022,10
4,1,10,Schuldendiensthilfen und Erstattungen von Verw...,2.000,261,0,2022,10


In [13]:
nrw_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8832 entries, 0 to 8831
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ep                   8832 non-null   int64 
 1   kapitel              8832 non-null   int64 
 2   zweckbestimmung      8832 non-null   object
 3   ansatz_2022_in_euro  8832 non-null   object
 4   gruppe               8832 non-null   int64 
 5   counter              8832 non-null   int64 
 6   jahr                 8832 non-null   int64 
 7   state_id             8832 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 552.1+ KB


In [14]:
nrw_2022['ep'] = nrw_2022['ep'].astype(str)
nrw_2022['kapitel'] = nrw_2022['kapitel'].astype(str)
#sh_2022['titel'] = sh_2022['titel'].astype(str)
#sh_2022['titel'] = sh_2022['titel'].apply(lambda x:x.zfill(5))
nrw_2022['ep'] = nrw_2022['ep'].apply(lambda x:x.zfill(2))

nrw_2022['kapitel'] = nrw_2022['kapitel']#.str#[-2:]
nrw_2022

,ep,kapitel,zweckbestimmung,ansatz_2022_in_euro,gruppe,counter,jahr,state_id
0,01,10,Vermischte Einnahmen,20.000,119,1,2022,10
1,01,10,Mieten und Pachten,10.000,124,1,2022,10
2,01,10,Erstattung von Verwaltungsausgaben durch die L...,0,232,0,2022,10
3,01,10,Erstattungen von Verwaltungsausgaben von Sozia...,0,236,0,2022,10
4,01,10,Schuldendiensthilfen und Erstattungen von Verw...,2.000,261,0,2022,10
...,...,...,...,...,...,...,...,...
8827,20,900,Zusätzliche Alters- und Hinterbliebenenversorg...,0,636,20,2022,10
8828,20,900,Erstattungen von Versorgungsbezügen an Zweckve...,10.000,637,0,2022,10
8829,20,900,Sonstige Erstattungen von Versorgungsbezügen,10.000,671,0,2022,10
8830,20,900,"Zuführung an das Sondervermögen ""Pensionsfonds...",200.000.000,919,10,2022,10


In [15]:
nrw_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8832 entries, 0 to 8831
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ep                   8832 non-null   object
 1   kapitel              8832 non-null   object
 2   zweckbestimmung      8832 non-null   object
 3   ansatz_2022_in_euro  8832 non-null   object
 4   gruppe               8832 non-null   int64 
 5   counter              8832 non-null   int64 
 6   jahr                 8832 non-null   int64 
 7   state_id             8832 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 552.1+ KB
